<a href="https://colab.research.google.com/github/bud-primordium/pytorch_exercise/blob/main/buildmodel_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| **Build Model** \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Build the Neural Network
========================

Neural networks comprise of layers/modules that perform operations on
data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace
provides all the building blocks you need to build your own neural
network. Every module in PyTorch subclasses the
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules
(layers). This nested structure allows for building and managing complex
architectures easily.

In the following sections, we\'ll build a neural network to classify
images in the FashionMNIST dataset.


In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
=======================

We want to be able to train our model on an
[accelerator](https://pytorch.org/docs/stable/torch.html#accelerators)
such as CUDA, MPS, MTIA, or XPU. If the current accelerator is
available, we will use it. Otherwise, we use the CPU.


In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


Define the Class
================

We define our neural network by subclassing `nn.Module`, and initialize
the neural network layers in `__init__`. Every `nn.Module` subclass
implements the operations on input data in the `forward` method.


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`,
and print its structure.


In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model\'s
`forward`, along with some [background
operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0
corresponding to each output of 10 raw predicted values for each class,
and dim=1 corresponding to the individual values of each output. We get
the prediction probabilities by passing it through an instance of the
`nn.Softmax` module.


In [16]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6], device='cuda:0')


In [17]:
print(logits)

tensor([[-0.0035, -0.1315, -0.0037,  0.0356, -0.1314,  0.0392,  0.0416, -0.0160,
         -0.0101, -0.0343]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [14]:
print(model.linear_relu_stack[0].weight)
print(model.linear_relu_stack[0].bias)

Parameter containing:
tensor([[-0.0164, -0.0342, -0.0328,  ..., -0.0346,  0.0149,  0.0327],
        [-0.0068,  0.0188,  0.0317,  ..., -0.0232,  0.0021,  0.0123],
        [-0.0057,  0.0309,  0.0175,  ...,  0.0357, -0.0086,  0.0176],
        ...,
        [-0.0096,  0.0220,  0.0346,  ..., -0.0223,  0.0187, -0.0343],
        [-0.0079, -0.0050,  0.0074,  ..., -0.0003, -0.0193, -0.0183],
        [ 0.0229, -0.0074, -0.0078,  ...,  0.0056, -0.0064, -0.0309]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-3.0261e-02,  2.6911e-02,  3.1774e-02, -2.5712e-02, -9.4018e-03,
        -4.1307e-04,  3.4679e-02,  2.4243e-02, -3.1599e-03,  8.7204e-03,
        -3.0754e-03, -1.3899e-02, -3.5596e-02, -1.5333e-02, -3.2874e-02,
        -2.0687e-02,  9.8735e-03, -2.7013e-02,  2.1065e-02,  2.5568e-02,
         3.1003e-02,  1.5670e-02, -3.7678e-03,  1.3906e-02, -2.2742e-02,
         4.5000e-03,  3.2461e-02, -1.3670e-02,  2.1341e-02,  1.9879e-03,
        -1.8148e-02,  2.5944e-02,  1.085

In [15]:
# 计算第一层权重的均值和标准差
weight = model.linear_relu_stack[0].weight.data
print(f"Mean: {weight.mean().item()}, Std: {weight.std().item()}")


Mean: -2.436938666505739e-05, Std: 0.020618092268705368


------------------------------------------------------------------------


Model Layers
============

Let\'s break down the layers in the FashionMNIST model. To illustrate
it, we will take a sample minibatch of 3 images of size 28x28 and see
what happens to it as we pass it through the network.


In [18]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
==========

We initialize the
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784
pixel values ( the minibatch dimension (at dim=0) is maintained).


In [19]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear
=========

The [linear
layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its
stored weights and biases.


In [20]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
=======

Non-linear activations are what create the complex mappings between the
model\'s inputs and outputs. They are applied after linear
transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use
[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
between our linear layers, but there\'s other activations to introduce
non-linearity in your model.


In [21]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.4812,  0.1753, -0.0503, -0.0217,  0.1062, -0.3995, -0.1282, -0.1224,
          0.1303, -0.2628, -0.0568,  0.2910,  0.6895, -0.2325,  0.4343,  0.4368,
          0.3807,  0.6573, -0.2607, -0.2504],
        [-0.1404,  0.2778, -0.2507, -0.1662,  0.3757, -0.0464, -0.1460,  0.0906,
         -0.1820, -0.4148,  0.2344,  0.5214,  0.4365,  0.0109,  0.1321,  0.5000,
          0.3920,  0.5728, -0.0631, -0.3561],
        [-0.5022,  0.3529, -0.4123, -0.2893, -0.0735, -0.3184,  0.0121,  0.2385,
          0.0726, -0.4496,  0.1570,  0.4704,  0.2880,  0.1276, -0.0174,  0.0675,
          0.2984,  0.3317, -0.2588, -0.4140]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.1753, 0.0000, 0.0000, 0.1062, 0.0000, 0.0000, 0.0000, 0.1303,
         0.0000, 0.0000, 0.2910, 0.6895, 0.0000, 0.4343, 0.4368, 0.3807, 0.6573,
         0.0000, 0.0000],
        [0.0000, 0.2778, 0.0000, 0.0000, 0.3757, 0.0000, 0.0000, 0.0906, 0.0000,
         0.0000, 0.2344, 0.5214, 0.4365, 0.0109, 0.13

nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
is an ordered container of modules. The data is passed through all the
modules in the same order as defined. You can use sequential containers
to put together a quick network like `seq_modules`.


In [22]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax
==========

The last linear layer of the neural network returns [logits]{.title-ref}
- raw values in \[-infty, infty\] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)
module. The logits are scaled to values \[0, 1\] representing the
model\'s predicted probabilities for each class. `dim` parameter
indicates the dimension along which the values must sum to 1.


In [23]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters
================

Many layers inside a neural network are *parameterized*, i.e. have
associated weights and biases that are optimized during training.
Subclassing `nn.Module` automatically tracks all fields defined inside
your model object, and makes all parameters accessible using your
model\'s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and
a preview of its values.


In [24]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0164, -0.0342, -0.0328,  ..., -0.0346,  0.0149,  0.0327],
        [-0.0068,  0.0188,  0.0317,  ..., -0.0232,  0.0021,  0.0123]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0303,  0.0269], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0304,  0.0341,  0.0006,  ..., -0.0204,  0.0363, -0.0362],
        [-0.0032,  0.0134,  0.0303,  ..., -0.0257, -0.0116, -0.0270]],
       device='cuda:0', grad_fn=<Sl

------------------------------------------------------------------------


Further Reading
===============

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)
